In [1]:
# !pip install ipywidgets (if needed)
# Importing necessary modules.
import numpy as np
import pandas as pd
import ipywidgets as ipy
pd.options.display.float_format = '{0:,.2f}'.format
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (18, 9)
sns.set_style('darkgrid')
import matplotlib.ticker as tt

In [2]:
# Loading all files and dropped some of the unnecessary columns.
games_data = pd.read_csv("games.csv").drop(['GAME_ID','GAME_STATUS_TEXT','FG_PCT_home','FT_PCT_home','FG3_PCT_home',
                                     'FG_PCT_away','FG3_PCT_away','FT_PCT_away','HOME_TEAM_WINS','TEAM_ID_home','TEAM_ID_away'],axis=1)

games_data_homeaway = pd.read_csv("games.csv").drop(['GAME_DATE_EST','GAME_ID','GAME_STATUS_TEXT','HOME_TEAM_WINS','TEAM_ID_home','TEAM_ID_away'],axis=1)

players_data = pd.read_csv("players.csv")

players_bios = pd.read_csv("players_bios.csv").drop(['Birth_City'],axis=1)

finals_data = pd.read_excel('NBA Finals and MVP.xlsx').drop(['MVP Nationality','MVP Position','MVP Height (m)','Final Sweep ?','Result'],axis=1)

teams_info = pd.read_csv('teams_info.csv').drop(['ARENA','OWNER','GENERALMANAGER','HEADCOACH','ABBREVIATION'],axis=1)

daily_points_table_first = pd.read_csv('ranking.csv').drop(['LEAGUE_ID','RETURNTOPLAY'],axis=1)
# Here I filtered out Season_ID 22002 and 22020 because they have incomplete data.
daily_points_table = daily_points_table_first[(daily_points_table_first['Season_ID']>22002) & (daily_points_table_first['Season_ID']!=22020)]
 
games_details = pd.read_csv('games_details.csv').drop(['FG_PCT','FG3_PCT','FT_PCT','OREB','DREB','PLUS_MINUS'],axis=1)

In [3]:
# LA clippers, New orleans/Oklahome City and Seattle teams have changed their names so I replaced their old teams names with new teams names.
la_clippers = daily_points_table[daily_points_table['Team Name']=='LA Clippers']
la_clippers['Team Name'] = 'L.A. Clippers'
no_oklahoma = daily_points_table[daily_points_table['Team Name']=='New Orleans/Oklahoma City']
no_oklahoma['Team Name'] = 'New Orleans'
seattle = daily_points_table[daily_points_table['Team Name']=='Seattle']
seattle['Team Name'] = 'Oklahoma City'

combined_teams_df = la_clippers.append([no_oklahoma,seattle,daily_points_table])
daily_points_table = combined_teams_df[(combined_teams_df['Team Name']!='LA Clippers') & (combined_teams_df['Team Name']!='New Orleans/Oklahoma City') & (combined_teams_df['Team Name']!='Seattle')]

## Question 1 : From NBA Champions between 1947 and 2020, Find out how many times each team won NBA Championship.

#### I have merged teams_info which have information about all 30 teams and finals_data which is the NBA finals result. Then, I grouped them by 'Champion Name' column so that I could find how many times each team won NBA Championship.

In [4]:
champions_data = finals_data.merge(teams_info,left_on='Champion Name',right_on='Team Name')[['Year','NBA Champion','Champion Name','NBA Vice-Champion','Year Founded']]
no_of_champions = champions_data.groupby('Champion Name').count()['NBA Champion'].sort_values(ascending=False).to_frame().rename(columns={'NBA Champion':'No. of Championships'})
no_of_champions

,No. of Championships
Champion Name,
Lakers,17
Celtics,17
Bulls,6
Warriors,6
Spurs,5
Pistons,3
Heat,3
76ers,3
Knicks,2


## Question 2 : Which teams are still to win their first NBA Championship? By checking their foundation year, we can imagine for how long their first championship is due.

#### First I created a column named 'Years Taken to Win First NBA Championship', then I grouped champions_data by 'Champion Name' column which gave me NBA winners list. At last, I applied '~' operator which gives me list of teams which have not won single NBA title.

In [5]:
champions_data['Years Taken to Win First NBA Championship'] = champions_data['Year'] - champions_data['Year Founded']
winners_list = champions_data.groupby('Champion Name').min('Years Taken to Win First NBA Championship').index
not_won_teams = teams_info [~teams_info.isin(winners_list)].drop(['TEAM_ID'],axis=1).dropna().set_index('Team Name').sort_values('Year Founded')
not_won_teams

,Year Founded,City
Team Name,,
Suns,1968,Phoenix
Clippers,1970,Los Angeles
Jazz,1974,Utah
Nuggets,1976,Denver
Nets,1976,Brooklyn
Pacers,1976,Indiana
Hornets,1988,Charlotte
Timberwolves,1989,Minnesota
Magic,1989,Orlando


## Question 3 : How many years each team took to win its first NBA championship since it was founded? Also, I have sorted them so that we could find the teams which were quickest to win NBA Championship.

#### I already created 'Years Taken to Win First NBA Championship' column in my champions_data dataframe, so I just used that column and sort the values which gave me the list of teams which were quickest to win first NBA championship since it was founded.

In [6]:
quickest_champions = champions_data.groupby('Champion Name').min('Years Taken to Win First NBA Championship').sort_values('Years Taken to Win First NBA Championship')
quickest_champions_final = quickest_champions[['Year Founded','Year','Years Taken to Win First NBA Championship']].rename(columns={'Year':'First Championship Year'})
quickest_champions_final

,Year Founded,First Championship Year,Years Taken to Win First NBA Championship
Champion Name,,,
Lakers,1948,1949,1
Warriors,1946,1947,1
Bucks,1968,1971,3
Kings,1948,1951,3
76ers,1949,1955,6
Trail Blazers,1970,1977,7
Hawks,1949,1958,9
Celtics,1946,1957,11
Thunder,1967,1979,12


#### This function returns points table for the given conference at the end of each year's NBA regular season. I have daily points table for each season for each conference, so I just used max function to find out maximum date which is the season ending date.

In [7]:
def common_fun(x):
    regular_season_points_table = daily_points_table[daily_points_table.Conference==x]
    final_points_table = regular_season_points_table.groupby('Season_ID')['Standings Date'].max().to_frame().merge(regular_season_points_table)
    return final_points_table

## I have used ipy.Dropdown for the dropdown menu and ipy.interact for making my code interactive in all the functions.

## Question 4 : Between 2003 and 2020, list out teams from each conference which finished first in their conference and won NBA championship that year.

#### I used common_fun which returned me points table for each season. Then, I only selected teams which finished first in their conference in that year and then I appended that data into empty_df.  Thereafter I merged final_table_toppers with teams_info and then I compared top team with that year's champion name so it gave the list of teams from each conference which finished first in their conference and won NBA that year.

In [8]:
def table_topper_won(x):
    empty_df = pd.DataFrame(columns = ['Team_ID','Season_ID','Team Name','G','W','L','W_PCT'])
    conf_data = common_fun(x)

    for i in conf_data.Season_ID.unique():
        zz = conf_data[conf_data.Season_ID==i].iloc[0].to_frame().T
        empty_df = empty_df.append(zz)
    
    empty_df['Year'] = empty_df['Season_ID'] - 20000 + 1
    final_table_toppers = empty_df.merge(finals_data)[['Year','Western Champion','Eastern Champion','Champion Name','Team_ID']]
    
    top_teams_performance = final_table_toppers.merge(teams_info,left_on='Team_ID',right_on='TEAM_ID').set_index('Year').drop(['Year Founded','City','Team_ID','TEAM_ID'],axis=1)
    top_teams_performance.rename(columns={'Team Name':'Table Topper Team'},inplace=True)
    top_teams_performance['Top_team_winning'] = np.where(top_teams_performance['Champion Name'] == top_teams_performance['Table Topper Team'],'Yes','No')
    sorted_top_performance = top_teams_performance.sort_values('Year')
    display(sorted_top_performance[sorted_top_performance.Top_team_winning=='Yes'])
    
conf_selector_table_topper_won = ipy.Dropdown(
options=['West','East'],
description='Conference')

ipy.interact(table_topper_won, x = conf_selector_table_topper_won)

interactive(children=(Dropdown(description='Conference', options=('West', 'East'), value='West'), Output()), _…

<function __main__.table_topper_won(x)>

In [9]:
# I have players data between 2003-2020. So, I have only kept dataset of players whose Birth_Year>=1960.
current_players_bios = players_bios[players_bios.Birth_Year>=1960]

# I have information about all players whether they played in a game or not. So, I kept only players who played the game by considering 'MIN' column.
lineup_players = games_details[games_details.MIN>'00:00:00']
no_of_games_played_df = lineup_players.groupby('Player Name')['Game_ID'].count().to_frame().rename(columns={'Game_ID':'No. of Games Played'}).sort_values('No. of Games Played',ascending=False)

# For the next question, I only kept players who played at least 300 matches so that it can give me some reliable correlation.
minimum_matches_df = no_of_games_played_df[no_of_games_played_df['No. of Games Played'] >=300]
players_matches_data = lineup_players.merge(minimum_matches_df,left_on='Player Name',right_on=minimum_matches_df.index)

In [10]:
# Here I calculated free throw percentage for the players. 
# Also, I megred free_throw_data and current_players_bios so that both Free Throw Percentage and Players Height columns will be in the same dataframe.
free_throw_data = players_matches_data.groupby('Player Name').sum()[['FTM','FTA']]
free_throw_data['FT_PCT'] = free_throw_data['FTM']*100/ free_throw_data['FTA']

ft_height_merged_df = free_throw_data.merge(current_players_bios,left_on=free_throw_data.index,right_on='Player').sort_values(by='FT_PCT',ascending=False)

## Question 5 : Find out Correlation between Free Throw Percentage and Players' Height for the players who played at least 300 matches during 2003-2020.

#### Here I found out Pearson Correlation between Free Throw Percentage and Players' Height.

In [11]:
ft_height_merged_df.drop(['Weight','College','Birth_Year','Birth_State'],axis=1).set_index('Player')
corr = ft_height_merged_df['FT_PCT'].corr(ft_height_merged_df['Height'])
print('The Pearson Correlation between Free Throw Percentage and Player''s Height is corr',corr)

The Pearson Correlation between Free Throw Percentage and Players Height is corr -0.4169500032891986


### <u>Conclusion</u> : Pearson correlation between Free Throw Percentage and Players' Height is -0.42. Therefore, it is clear that  Free Throw Percentage and  Players' Height are not strongly correlated.

## Question 6 : LeBron James played for 3 teams during 2003-2020: Miami Heat, Cleveland Cavaliers & Los Angeles Lakers and he won NBA championship with all the teams. So, I compared his performance per game for these teams and also made a plot of comparison.

In [12]:
# I selected 'LeBron James' data to compare his performance for 3 teams.
lebron_data = lineup_players[lineup_players['Player Name']=='LeBron James'].drop(['Player Name','Team City','Comment','Game_ID','Team_ID','Player_ID','Start_Position','PF'],axis=1).set_index('Team_Abbreviation')

In [13]:
# I converted 'MIN' column which was in the form of "mm:ss" into total minutes played for all games.
lebron_time_data = lebron_data[~lebron_data['MIN'].str.contains(':')].rename(columns={'MIN':'m'})
lebron_time_data['m'] = lebron_time_data['m'].astype(int)
lebron_time_data['s'] = 0

lebron_time_data_mins = lebron_data[lebron_data['MIN'].str.contains(':')]
lebron_time_data_mins[['m','s']] = lebron_time_data_mins['MIN'].astype(str).str.split(':', expand=True).astype(int)
lebron_final_df = lebron_time_data_mins.append(lebron_time_data)
lebron_final_df['Minutes Played'] = lebron_final_df['m'] + (lebron_final_df['s']/60)
lebron_final_df.drop(['MIN','m','s'],axis=1,inplace=True)

In [14]:
# I found per game data for LeBron James for all 3 teams. Also, I found how many matches LeBron played for each team.
lebron_totals_by_team = lebron_final_df.groupby(lebron_final_df.index).mean()
players_matches_per_team = lineup_players.groupby(['Player Name','Team_Abbreviation']).count().sort_values(by='MIN',ascending=False)['Game_ID'].to_frame()
lebron_matches_per_team = (players_matches_per_team.loc[['LeBron James']]).reset_index(level=0, drop=True)

#### I included 'Seasons Played' and 'No. of times team Won NBA' columns into lebron_merged_df. Then, I found out percentage values for FG, FT and FG3. Then, I plotted LeBron's performance comparison for per game for CLE, MIA and LAL.

In [15]:
def lebron_fun(x):
    lebron_merged_df = lebron_matches_per_team.merge(lebron_totals_by_team,left_index=True,right_index=True)
    lebron_merged_df ['Seasons Played'] = [11,4,2]
    lebron_merged_df ['No. of times team Won NBA'] = [1,2,1]

    lebron_merged_df['FG_PCT'] = lebron_merged_df['FGM']*100/lebron_merged_df['FGA']
    lebron_merged_df['FT_PCT'] = lebron_merged_df['FTM']*100/lebron_merged_df['FTA']
    lebron_merged_df['FG3_PCT'] = lebron_merged_df['FG3M']*100/lebron_merged_df['FG3A']
    lebron_merged_df.drop(['FGM','FGA','FG3M','FG3A','FTM','FTA'],axis=1,inplace=True)
    lebron_merged_df.rename(columns={'Game_ID':'Games Played'},inplace=True)
    
    display(lebron_merged_df[[x]])
    ax = lebron_merged_df.plot(kind='bar',y=x,width=0.4)
    ax.set_xlabel('Team Name',fontsize=20)
    ax.set_ylabel('{}'.format(x),fontsize=20)
    ax.set_title('{} Comparison for LeBron James for all 3 teams'.format(x),fontsize=25)
    ax.set_xticklabels(labels=lebron_merged_df.index,fontsize=15,rotation=0)

conf_selector_lebron_fun = ipy.Dropdown(
options = np.sort(['Games Played', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PTS',
       'Minutes Played', 'Seasons Played', 'No. of times team Won NBA', 'FG_PCT', 'FT_PCT',
       'FG3_PCT']),
description='Category')
    
ipy.interact(lebron_fun, x = conf_selector_lebron_fun)

interactive(children=(Dropdown(description='Category', options=('AST', 'BLK', 'FG3_PCT', 'FG_PCT', 'FT_PCT', '…

<function __main__.lebron_fun(x)>

## Triple Double means a player scored at least 10 points (double digit) in at least 3 of the 5 statistical categories: Points, Rebounds, Steals, Assists and Blocks.

#### I made a copy of lineup_players so the changes I made will not affect the original lineup_players dataframe. As per Triple Double definition I checked all 5 columns whether they have a double digit value or not. If they have double digit value then I put 1 otherwise 0.

In [16]:
lineup_players_copy = lineup_players
lineup_players_copy['PTS_TD']=np.where(lineup_players_copy['PTS']>=10,1,0)
lineup_players_copy['REB_TD']=np.where(lineup_players_copy['REB']>=10,1,0)
lineup_players_copy['STL_TD']=np.where(lineup_players_copy['STL']>=10,1,0)
lineup_players_copy['AST_TD']=np.where(lineup_players_copy['AST']>=10,1,0)
lineup_players_copy['BLK_TD']=np.where(lineup_players_copy['BLK']>=10,1,0)

## Question 7 : Between 2003 and 2020, List out top 10 players with most no. of Triple Doubles.

### After summation, if any player have 'TD' column value 3 or more then it means he scored Triple Double in that game and then I counted such players and their total no. of triple doubles.

In [17]:
lineup_players_copy['TD'] = lineup_players_copy['PTS_TD']+lineup_players_copy['REB_TD']+lineup_players_copy['STL_TD']+lineup_players_copy['AST_TD']+lineup_players_copy['BLK_TD']
lineup_players_copy['TD?'] = np.where(lineup_players_copy['TD']>=3,True,False)
triple_double_df = lineup_players_copy[lineup_players_copy['TD?']==True]
triple_double_df.groupby('Player Name')['Game_ID'].count().sort_values(ascending=False).to_frame().rename(columns={'Game_ID':'No. of Triple Doubles'}).head(10)

,No. of Triple Doubles
Player Name,
Russell Westbrook,171
LeBron James,127
Jason Kidd,62
James Harden,60
Nikola Jokic,58
Rajon Rondo,42
Draymond Green,37
Luka Doncic,36
Ben Simmons,34


In [18]:
# I calculated all players' per game data across all statistical categories. Also, I found the percentage values for FG, FT and FG3.
players_avg_data = lineup_players.groupby('Player Name').mean().drop(['Game_ID','Team_ID','Player_ID'],axis=1)
players_avg_data['FG_PCT'] = players_avg_data['FGM'] *100/players_avg_data['FGA']
players_avg_data['FT_PCT'] = players_avg_data['FTM']*100/players_avg_data['FTA']
players_avg_data['FG3_PCT'] = players_avg_data['FG3M']*100/players_avg_data['FG3A']

players_avg_df_final = players_avg_data.drop(['FGM','FGA','FG3M','FG3A','FTM','FTA','PTS_TD','REB_TD','STL_TD','AST_TD','BLK_TD','TD','TD?'],axis=1).fillna(0).T

## Question 8 : Comparison between 2 selected players for all the statistical categories. This is per game's performance comparison.

#### Here I created function which compares two players' per game performance using players_avg_df_final dataframe.

In [19]:
def player_comp_avg_fun(x,y):
    players_avg_int = players_avg_df_final[[x,y]]
    display(players_avg_int)
    
player1_comp_avg_fun_selector = ipy.Dropdown(
options = np.sort(players_avg_df_final.columns),
description = 'Player 1')

player2_comp_avg_fun_selector  = ipy.Dropdown(
options = np.sort(players_avg_df_final.columns),
description = 'Player 2')

ipy.interact(player_comp_avg_fun,x=player1_comp_avg_fun_selector, y=player2_comp_avg_fun_selector )

interactive(children=(Dropdown(description='Player 1', options=('A.J. Guyton', 'AJ Hammons', 'AJ Price', 'Aaro…

<function __main__.player_comp_avg_fun(x, y)>

In [20]:
# I seperated Home wins, Home Loses, Away Wins, Away Loses from HOME_RECORD and ROAD_RECORD column of daily_points_table.
# Also, I converted all 4 columns into numeric integer values.
daily_points_table[['Home Wins','Home Loses']] = daily_points_table.HOME_RECORD.str.split("-",expand=True)
daily_points_table[['Away Wins','Away Loses']] = daily_points_table.ROAD_RECORD.str.split("-",expand=True)

daily_points_table['Away Wins'] = pd.to_numeric(daily_points_table['Away Wins'])
daily_points_table['Home Wins'] = pd.to_numeric(daily_points_table['Home Wins'])
daily_points_table['Away Loses'] = pd.to_numeric(daily_points_table['Away Loses'])
daily_points_table['Home Loses'] = pd.to_numeric(daily_points_table['Home Loses'])

daily_points_table.drop(['HOME_RECORD','ROAD_RECORD'],axis=1,inplace=True)

## Question 9 : Between 2003 and 2020 which team had most no. of Home Wins? Also, which team had most no. of Away Wins? List out Top 10 teams from each conference.

#### To find out this, I have used 'Home Wins' and 'Away Wins' columns which I have created earlier. Then, sorted it in descending order to find out top 10 teams from each conference.

In [21]:
def best_home_away(x):    
    final_points_table = common_fun(x)
    home_wins_top10 = final_points_table.groupby('Team Name')['Home Wins'].sum().sort_values(ascending=False).head(10).to_frame()
    away_wins_top10 = final_points_table.groupby('Team Name')['Away Wins'].sum().sort_values(ascending=False).head(10).to_frame()
    display(home_wins_top10) 
    display(away_wins_top10)

conf_selector_best_home_away = ipy.Dropdown(
options=['West','East'],
description='Conference')

ipy.interact(best_home_away,x = conf_selector_best_home_away)

interactive(children=(Dropdown(description='Conference', options=('West', 'East'), value='West'), Output()), _…

<function __main__.best_home_away(x)>

## Question 10 : List out teams from every year's points table from each conference which qualified for playoffs during 2004-2020.

#### In NBA, top 8 teams from each conference's points table qualifies every year. So, I used common_fun which returned me the points table for selected conference, from which I selected top 8 teams. Then, I used empty_df2 dataframe to merge it with conference_most_qualificaition dataframe.

In [22]:
def no_of_qual(y,z):
    empty_df2 = pd.DataFrame(columns = ['Season_ID','Team Name','G','W','L','W_PCT'])
    conference_most_qualificaition = common_fun(y)
    for i in conference_most_qualificaition.Season_ID.unique():
        ww = conference_most_qualificaition[conference_most_qualificaition.Season_ID==i].nlargest(8,'W')
        empty_df2 = empty_df2.append(ww)
    empty_df2['Year'] = empty_df2['Season_ID'] - 20000 + 1
    empty_df2.drop(['Season_ID','Standings Date','Team_ID','Conference'],axis=1,inplace=True)
    empty_df2.set_index('Team Name',inplace=True)
    display(empty_df2[empty_df2.Year==z])
    
conf_selector_no_of_qual = ipy.Dropdown(
options = ['West','East'],
description = 'Conference')

year_selector_no_of_qual = ipy.Dropdown(
options = np.arange(2004,2021,1),
description = 'Year')

ipy.interact(no_of_qual,y = conf_selector_no_of_qual, z = year_selector_no_of_qual)

interactive(children=(Dropdown(description='Conference', options=('West', 'East'), value='West'), Dropdown(des…

<function __main__.no_of_qual(y, z)>

## Question 11 : Between 2004 and 2020, List out top 10 teams from each conference which qualified most for the play-offs.

#### To find out this, I have created an empty dataframe to append, also I used common_fun function which returned points table. Top 8 teams qualifies for the play-offs, so I filtered top 8 teams from conference_most_qualificaition. If the team qualified for the play-offs then I added value 1 in the column 'Play-offs Qualification Times' so that I can count total no. of play-offs qualifications.

In [23]:
def most_qual(x):
    empty_df3 = pd.DataFrame(columns = ['Season_ID','Team Name','G','W','L','W_PCT'])
    conference_most_qualificaition = common_fun(x)
    for i in conference_most_qualificaition.Season_ID.unique():
        ww = conference_most_qualificaition[conference_most_qualificaition.Season_ID==i].nlargest(8,'W')
        empty_df3 = empty_df3.append(ww)
    empty_df3['Play-offs Qualification Times'] = 1
    
    display(empty_df3.groupby('Team Name').count()['Play-offs Qualification Times'].sort_values(ascending=False).to_frame().head(10))

conf_selector_most_qual = ipy.Dropdown(
options=['West','East'],
description='Conference')

ipy.interact(most_qual, x = conf_selector_most_qual)

interactive(children=(Dropdown(description='Conference', options=('West', 'East'), value='West'), Output()), _…

<function __main__.most_qual(x)>

#### Here in games_data_homeaway SEASON 2018 represents 2018-19 data, so I give them a new column named 'Year' which is SEASON+1. In games_data_homeaway, I only have IDs of home teams and visitor(away) teams. So, I performed merge operations twice to assign home teams and visitor teams' names to their corresponding IDs.

In [24]:
games_data_homeaway['Year'] = games_data_homeaway.SEASON + 1
teams_info_filtered = teams_info[['TEAM_ID','Team Name']]
home_team_incl = games_data_homeaway.merge(teams_info_filtered,left_on='HOME_TEAM_ID',right_on='TEAM_ID').rename(columns={'Team Name':'Home Team'})
final_homeaway_df = home_team_incl.merge(teams_info_filtered,left_on='VISITOR_TEAM_ID',right_on='TEAM_ID').rename(columns={'Team Name':'Away Team'})

final_homeaway_df.drop(['HOME_TEAM_ID','VISITOR_TEAM_ID','SEASON','TEAM_ID_x','TEAM_ID_y'],axis=1,inplace=True)
# Also, I removed year 2021's data as 2020-21 NBA season is not finished yet, it is still ongoing.
final_homeaway_finished = final_homeaway_df[final_homeaway_df.Year!=2021]

## Question 12 : Plot the comparison of all teams' average (per game) performance in any statistical category for the selected year between 2004 and 2020.

#### Here I created 2 dataframes one for home team's average stats and other for away team's average stats, here I grouped the dataset by 'Home Team' as it is teamwise comparison. Then, I merged both of the dataframes so that I can create a dataframe which have combined average data. At last, I plotted all teams' performance in any statistical category for the selected year.

In [25]:
def teamwise_home_away(x,y):
    final_homeaway_year = final_homeaway_finished[final_homeaway_finished.Year==x]
    final_df_teams_home = final_homeaway_year.groupby('Home Team').mean().drop(['PTS_away','FG_PCT_away','FT_PCT_away','FG3_PCT_away','AST_away','REB_away','Year'],axis=1)
    final_df_teams_home['Average Home FG Percentage'] = final_df_teams_home['FG_PCT_home']*100
    final_df_teams_home['Average Home FT Percentage'] = final_df_teams_home['FT_PCT_home']*100
    final_df_teams_home['Average Home 3-Pointers Percentage'] = final_df_teams_home['FG3_PCT_home']*100
    final_df_teams_home.rename(columns={'AST_home':'Average Home Assists','REB_home':'Average Home Rebounds','PTS_home':'Average Home Points'},inplace=True)

    final_df_teams_away = final_homeaway_year.groupby('Away Team').mean().drop(['PTS_home','FG_PCT_home','FT_PCT_home','FG3_PCT_home','AST_home','REB_home','Year'],axis=1)
    final_df_teams_away['Average Away FG Percentage'] = final_df_teams_away['FG_PCT_away']*100
    final_df_teams_away['Average Away FT Percentage'] = final_df_teams_away['FT_PCT_away']*100
    final_df_teams_away['Average Away 3-Pointers Percentage'] = final_df_teams_away['FG3_PCT_away']*100
    final_df_teams_away.rename(columns={'AST_away':'Average Away Assists','REB_away':'Average Away Rebounds','PTS_away':'Average Away Points'},inplace=True)

    final_df_teams = final_df_teams_home.merge(final_df_teams_away,left_index=True,right_index=True)
    ax = final_df_teams.plot(kind='bar',y=y,width=0.8)
    ax.set_xlabel('Team Name',fontsize=20)
    ax.set_ylabel('{}'.format(y),fontsize=20)
    ax.set_title('{} Comparison Between All Teams For {}'.format(y,x),fontsize=25)
    ax.set_xticklabels(labels=final_df_teams.index,fontsize=15)
    ax.set_yticklabels(labels=[0,0,10,20,30,40,50,60,70,80,90,100,110,120,130],fontsize=15)
    ax.yaxis.set_major_locator(tt.MultipleLocator(10))
    
teamwise_home_away_year_selector = ipy.Dropdown(
options = np.arange(2004,2021,1),
description = 'Year')

teamwise_home_away_criteria_selector = ipy.Dropdown(
options = ['Average Home Points','Average Home Assists','Average Home Rebounds','Average Home FG Percentage','Average Home FT Percentage','Average Home 3-Pointers Percentage','Average Away Points','Average Away Assists','Average Away Rebounds','Average Away FG Percentage','Average Away FT Percentage','Average Away 3-Pointers Percentage'],
description = 'Category')

ipy.interact(teamwise_home_away ,x = teamwise_home_away_year_selector, y = teamwise_home_away_criteria_selector)

interactive(children=(Dropdown(description='Year', options=(2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 20…

<function __main__.teamwise_home_away(x, y)>

## Question 13 : Plot the side-by-side yearly comparison for home and away performance in any statistical category for the selected team between 2004 and 2020.

#### Here I created 2 dataframes one for home team's average stats and other for away team's average stats, here I grouped the dataset by 'Year' as it is yearly comparison. Then, I merged both of the dataframes so that I can create a dataframe which have combined average data. At last, I plotted side-by-side bar chart which shows yearly comparison for home and away performance in any statistical category for the selected team between 2004 and 2020.

In [26]:
def yearly_home_away_comp(x,y):
    yearly_df = final_homeaway_finished[final_homeaway_finished['Home Team']==x] 
    yearly_home_df = yearly_df.groupby('Year').mean().drop(['PTS_away','FG_PCT_away','FT_PCT_away','FG3_PCT_away','AST_away','REB_away'],axis=1)
    yearly_home_df['Average Home FG Percentage'] = yearly_home_df['FG_PCT_home']*100
    yearly_home_df['Average Home FT Percentage'] = yearly_home_df['FT_PCT_home']*100
    yearly_home_df['Average Home 3-Pointers Percentage'] = yearly_home_df['FG3_PCT_home']*100
    yearly_home_df.rename(columns={'AST_home':'Average Home Assists','REB_home':'Average Home Rebounds','PTS_home':'Average Home Points'},inplace=True)

    yearly_away_df = yearly_df.groupby('Year').mean().drop(['PTS_home','FG_PCT_home','FT_PCT_home','FG3_PCT_home','AST_home','REB_home'],axis=1)
    yearly_away_df['Average Away FG Percentage'] = yearly_away_df['FG_PCT_away']*100
    yearly_away_df['Average Away FT Percentage'] = yearly_away_df['FT_PCT_away']*100
    yearly_away_df['Average Away 3-Pointers Percentage'] = yearly_away_df['FG3_PCT_away']*100
    yearly_away_df.rename(columns={'AST_away':'Average Away Assists','REB_away':'Average Away Rebounds','PTS_away':'Average Away Points'},inplace=True)

    yearly_final_df = yearly_home_df.merge(yearly_away_df,left_index=True,right_index=True)
    
    # this is for side-by-side bar chart.
    fig = plt.figure()
    ax = fig.add_subplot(111)
    width=0.40
    x1 = ax.bar(yearly_final_df.index,yearly_final_df['Average Home {}'.format(y[8:])],width,color='goldenrod')
    x2 = ax.bar(yearly_final_df.index+width,yearly_final_df['Average Away {}'.format(y[8:])],width,color='teal')
    ax.set_xticks(yearly_final_df.index + width / 2)
    ax.set_xlabel('Year',fontsize=20)
    ax.set_ylabel(y,fontsize=20)
    # To increase font-size I gave values to the lables inside ax.set_xticklabels and ax.set_yticklabels.
    ax.set_xticklabels(labels=yearly_final_df.index,fontsize=15)
    ax.set_yticklabels(labels=[0,0,10,20,30,40,50,60,70,80,90,100,110,120,130],fontsize=15)
    ax.yaxis.set_major_locator(tt.MultipleLocator(10))
    ax.legend( (x1[0], x2[0]), ('Average Home {}'.format(y[8:]), 'Average Away {}'.format(y[8:])) )
    ax.set_title('Average Home and Away {} Yearly Comparison for {}'.format(y[8:],x),fontsize=25)
    
yearly_home_away_comp_team_selector = ipy.Dropdown(
options = np.sort(final_homeaway_finished['Home Team'].unique()),
description = 'Team')

yearly_home_away_comp_criteria_selector = ipy.Dropdown(
options = ['Average Points','Average Assists','Average Rebounds','Average FG Percentage','Average FT Percentage','Average 3-Pointers Percentage'],
description = 'Category')

ipy.interact(yearly_home_away_comp, x = yearly_home_away_comp_team_selector, y = yearly_home_away_comp_criteria_selector)

interactive(children=(Dropdown(description='Team', options=('76ers', 'Bucks', 'Bulls', 'Cavaliers', 'Celtics',…

<function __main__.yearly_home_away_comp(x, y)>

## Question 14 : Yearwise comparison for selected team in these 7 categories : Total Wins, Total Loses, Winning Percentage, Home Wins, Away Wins, Home Loses, Away Loses, Team Position. Plot a comparison bar chart.

#### I created 'Year' column from 'Season_ID' column. Then, I found Winning percentage for each team and team's finishing position in their corresponding conference. Then, I plotted a yearly comparison bar chart for selected team in the mentioned 7 categories.

In [27]:
def teams_wins_loses_yearly(x,y):
    # There were 14 teams in the Western Conference of NBA Season 2003-04, while for other seasons there were 15 teams.
    teams_win_lose = daily_points_table.groupby('Season_ID')['Standings Date'].max().to_frame().merge(daily_points_table)
    teams_win_lose['Year'] = teams_win_lose['Season_ID'] - 20000 + 1
    teams_win_lose['W_PCT'] = teams_win_lose['W_PCT']*100
    teams_win_lose.sort_values(by=['Conference','Year'],inplace=True)
    # There were 14 teams in the Western Conference of NBA Season 2003-04, while for other seasons there were 15 teams.
    # So, to give all the teams their finishing positions in the conference each year, I used numpy's concatenate method with multiple inputs of a and b.
    a = np.arange(1,15,1) 
    b = np.arange(1,16,1)
    teams_win_lose['Team Position'] = np.concatenate((b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,a,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b))
    teams_win_lose.rename(columns={'W':'Total Wins','L':'Total Loses','W_PCT':'Winning Percentage'},inplace=True)
    team_selected = teams_win_lose[teams_win_lose['Team Name']==x]
    
    ax = team_selected.plot(kind='bar',x='Year',y=y,width=0.8)
    ax.set_xlabel('Year',fontsize=20)
    # To increase font-size I gave values to the lables inside ax.set_xticklabels.
    ax.set_xticklabels(labels=team_selected['Year'],fontsize=15,rotation=0)
    ax.set_ylabel('{}'.format(y),fontsize=20)
    ax.set_title('Yearly Comparison of {}\'s {}'.format(x,y),fontsize=25)

teams_wins_loses_yearly_team_selector = ipy.Dropdown(
options = np.sort(daily_points_table['Team Name'].unique()),
description = 'Team')

teams_wins_loses_yearly_criteria_selector = ipy.Dropdown(
options = ['Total Wins','Total Loses','Winning Percentage','Home Wins','Away Wins','Home Loses','Away Loses','Team Position'],
description = 'Category')

ipy.interact(teams_wins_loses_yearly, x = teams_wins_loses_yearly_team_selector, y = teams_wins_loses_yearly_criteria_selector)

interactive(children=(Dropdown(description='Team', options=('Atlanta', 'Boston', 'Brooklyn', 'Charlotte', 'Chi…

<function __main__.teams_wins_loses_yearly(x, y)>

## Question 15 : Between 1947 and 2020, List out NBA MVPs whose team won the championship.

#### To find out this, I have used finals_data which have MVPs information.  Then I compared 'MVP Status' and 'Champion' columns to find out list of NBA MVPs whose team won the championship.

In [28]:
mvp_info = finals_data.dropna()[['Year','NBA Champion','MVP Name','MVP Team','MVP status']]
mvp_nba_champions = (mvp_info[mvp_info['MVP status'] == 'Champion']).set_index('Year').drop(['MVP status'],axis=1)
mvp_nba_champions

,NBA Champion,MVP Name,MVP Team
Year,,,
1957,Boston Celtics,B. Cousy,Boston Celtics
1961,Boston Celtics,B. Russell,Boston Celtics
1962,Boston Celtics,B. Russell,Boston Celtics
1963,Boston Celtics,B. Russell,Boston Celtics
1965,Boston Celtics,B. Russell,Boston Celtics
1967,Philadelphia 76ers,W. Chamberlain,Philadelphia 76ers
1970,New York Knicks,W. Reed,New York Knicks
1971,Milwaukee Bucks,K. AJabbar,Milwaukee Bucks
1980,Los Angeles Lakers,K. AJabbar,Los Angeles Lakers
